# Notebook to explore the tables created by dune_source using the default config in this repo
Before running this notebook, run the pipeline to create the tables in the duckdb database.:
```
python duckdb_pipeline.py
```
or 
```
python explicit_resources_example.py
```

After running the pipeline, the duckdb database will be populated with the following tables:
- dex_volume
- dex_volume_incremental
- y2k_price_data
- custom_sql (with incremental cursor)

Important: make sure to close the connection to the duckdb database after running the notebook, if you don't, the connection will be locked and you won't be able to run the pipeline again.

In [55]:
import duckdb
db = duckdb.connect("dune_source.duckdb")

In [56]:
# dex_volume
db.execute("select * from dune_queries.dex_volume order by _col1").fetchdf()

,project,_col1,usd_volume,_dlt_load_id,_dlt_id
0,synthetix,2025-01-28 00:00:00.000 UTC,3.431989e+03,1740632236.762181,BAJOKVWIXXcj1Q
1,alienbase,2025-01-28 00:00:00.000 UTC,1.488904e+05,1740632236.762181,Bb1ibfPdodj8Dw
2,clipper,2025-01-28 00:00:00.000 UTC,2.122265e+06,1740632236.762181,LofZ8Q3jw0SUjQ
3,swapr,2025-01-28 00:00:00.000 UTC,6.961598e+00,1740632236.762181,03BHZ40/3HT2nQ
4,sterling_finance,2025-01-28 00:00:00.000 UTC,1.902615e+01,1740632236.762181,xUY3BIasxeKh8Q
...,...,...,...,...,...
4161,wagmi,2025-02-26 00:00:00.000 UTC,4.551986e+07,1740632236.762181,ylf6K012lh+gMQ
4162,openocean,2025-02-26 00:00:00.000 UTC,4.054873e+06,1740632236.762181,YXrVX3+2/iWrIg
4163,glacier,2025-02-26 00:00:00.000 UTC,2.170201e+02,1740632236.762181,AAYG0EwG5npKSw
4164,iziswap,2025-02-26 00:00:00.000 UTC,9.688930e+05,1740632236.762181,c/M4Vc/OkP1IYg


In [57]:
# notice that the date column goes back to 2025-01-01 (starting_replication_value)
db.execute("select * from dune_queries.dex_volume_incremental order by date").fetchdf() 

,project,date,usd_volume,_dlt_load_id,_dlt_id
0,syncswap,2025-01-01 00:00:00.000 UTC,1.976149e+06,1740631592.581887,3ZMcfbdYdoBj2g
1,zipswap,2025-01-01 00:00:00.000 UTC,7.324603e+02,1740631592.581887,bqYvhVbvM/D8LQ
2,spookyswap,2025-01-01 00:00:00.000 UTC,1.096529e+06,1740631592.581887,BMhXXdNmD7lZeQ
3,integral,2025-01-01 00:00:00.000 UTC,3.763263e+06,1740631592.581887,3BDSvqjVfaX9DQ
4,mummy_finance,2025-01-01 00:00:00.000 UTC,1.835366e+03,1740631592.581887,959wpeYBycNoag
...,...,...,...,...,...
7987,neopin,2025-02-27 00:00:00.000 UTC,1.812329e+02,1740631592.581887,Iq32oJmiRYj8yQ
7988,agni,2025-02-27 00:00:00.000 UTC,9.727873e+05,1740631592.581887,JQuvSttaQcyrog
7989,kodiak,2025-02-27 00:00:00.000 UTC,3.831964e+06,1740631592.581887,W805jIPV3pecPg
7990,spartadex,2025-02-27 00:00:00.000 UTC,4.688706e+01,1740631592.581887,0fV1wrxx0FnWRw


In [58]:
# y2k_price_data (pass in Y2K as symbol to query param)
db.execute("select * from dune_queries.y2k_price_data").fetchdf()

,timestamp,year,month,blockchain,contract_address,symbol,decimals,price,_dlt_load_id,_dlt_id
0,2022-09-20 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000034,1740632236.762181,bCVUrak10e5fOQ
1,2022-09-21 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000034,1740632236.762181,B0jF7H1n6VNnGQ
2,2022-10-18 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740632236.762181,UdcQ9V4PlIBMIg
3,2022-11-06 00:00:00.000 UTC,2022,11,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740632236.762181,M/LE8lEu9JpNig
4,2022-12-27 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.872464,1740632236.762181,ZEJaOtgWbDXcMA
...,...,...,...,...,...,...,...,...,...,...
4459,2024-12-21 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.924233,1740632236.762181,gTw0geWZ49J9jw
4460,2024-12-27 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.575739,1740632236.762181,g+e6aCjLSRatPw
4461,2025-01-06 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.706050,1740632236.762181,Zfz10fEWX8CqKQ
4462,2025-01-27 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.382923,1740632236.762181,8hE4HGi1+AmB0w


In [59]:
# custom_sql
db.execute("select * from dune_queries.custom_sql order by timestamp").fetchdf()

,timestamp,blockchain,contract_address,symbol,price,_dlt_load_id,_dlt_id
0,2024-11-02 00:00:00.000 UTC,base,0xf1c94f50d0a44947cf453141537a89e8dbb79441,BRETT,1.042080e-11,1740631931.536364,3pubDL5OM5vDIQ
1,2024-11-03 00:00:00.000 UTC,base,0x16bedfa55b98014a8d3b2eff62ec6efe493d49d1,BRETT,4.722576e-08,1740631931.536364,KKPNIRYwEo0dfw
2,2024-11-04 00:00:00.000 UTC,base,0xb9328262a17d22432f21daae2e27747912dd049f,BRETT,8.239284e-11,1740631931.536364,lebMfUBp/1WnvQ
3,2024-11-05 00:00:00.000 UTC,base,0xf9e5599758734589f7429e2a590a735e7b7f970d,BRETT,1.756815e-06,1740631931.536364,0UOtdV8WQe6Nkw
4,2024-11-06 00:00:00.000 UTC,base,0x06bda4df1836fa4c5cb73371db69011670fd1d3a,BRETT,9.637723e-18,1740631931.536364,85LMF3PPMJ2JLg
...,...,...,...,...,...,...,...
111,2025-02-22 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,5.008125e-02,1740631931.536364,2WdyDEHJ/ByznA
112,2025-02-23 00:00:00.000 UTC,base,0xb53a0820565ac31512ea4bf1b7dfd25394344cdf,BRETT,9.322995e-08,1740631931.536364,3CrU45rmo+ofyg
113,2025-02-24 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,4.414943e-02,1740631931.536364,G0NqVzI6nXHgWQ
114,2025-02-25 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,3.898919e-02,1740631931.536364,URjgBaHBZRiiag


In [60]:
# close the connection
db.close()